<a href="https://colab.research.google.com/github/viksingh/TrendFollowingStrategies/blob/master/100_day_weekly_with_weekly_snapshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import glob
import os
import numpy as np
import sys
import time
#import pdb
start_time = time.time()


from google.colab import drive
drive.mount('/content/drive')
!{sys.executable} -m pip install PyDrive
!{sys.executable} -m pip install yfinance


import yfinance as yf
import yfinance as yft

# Check if index is going up

moneyforeachstock = 2500
amountSpent = 0
c_stoplossfactorindexdown = 0.80
c_stoplossindexup = 0.90
numberOfStocks = 0
totalStocksHeld = 0

totalSpent = 0
totalValue = 0
totalValueSell = 0
totalStopLossMoney = 0
indexPath = '/content/drive/My Drive/Colab Notebooks/data2/index.csv'
stockCombinedPath = '/content/drive/My Drive/Colab Notebooks/data2/stockscombined.csv'

dfindex = pd.read_csv(indexPath, header=0)
dfstocks = pd.read_csv(stockCombinedPath, header=0 )

# Create a dataframe to store currently bought stocks
# Fields : STOCK, BOUGHT DATE , BOUGHT PRICE , QTY, UPDATE DATE, LAST PRICE, RoC % , BOUGHTPRICE , UPDTPRICE
# May be add an entry for each week when this gets checked

def fround(f):
    return str(round(f,2))

def createqueryDate(d):
    return 'DATE' + '==' + "'" + d + "'"

def checkActive():
    return "ACTIVE == 'X'"

def checkStock(s):
     return 'STOCK' + '==' + "'" + s + "'"



#pdb.set_trace()

from datetime import date, timedelta

start_date = date(2019, 7, 5)
end_date = date(2020, 1, 10)

delta = timedelta(days=7)

companiesList = dfstocks.STOCK.unique()

dfStocksCurrentlyBought = pd.DataFrame(columns=['STOCK','PURCHASEDATE','PURCHASEPRICE','QTY','UPDATEDATE','UPDATEPRICE','MAXPRICE','ROC', 'ACTIVE'])

while start_date <= end_date:
    weektotalValue = 0
    weektotalValueAPI = 0
# Check if index is increasing / decreasing this week
    processingDate = start_date.strftime("%Y-%m-%d")
    dfNasdaqCheckIndexIncreasing = dfindex.query(createqueryDate(processingDate))
    if (dfNasdaqCheckIndexIncreasing.shape[0])> 0:
        isIndexIncreasing = dfNasdaqCheckIndexIncreasing['CROSS'].iloc[-1]
        print('Index : ' + str(processingDate) + ' - ' + str(isIndexIncreasing))

# Update to current weeks price

        stockWeekUpdatePriceList = dfStocksCurrentlyBought.query("ACTIVE=='X'").STOCK.unique()
        for stockUpdatePrice in stockWeekUpdatePriceList:
            queryGetCurrentStockUpdatePrice = 'STOCK' + '==' + "'" + stockUpdatePrice + "'"
            dfCurrentstockUpdatePrice = dfstocks.query(queryGetCurrentStockUpdatePrice)

            queryGetUpdatePriceDateValue = "DATE" + "==" + "'" + processingDate + "'"
            dfCurrentstockUpdatePrice = dfCurrentstockUpdatePrice.query(queryGetUpdatePriceDateValue)
            dfCurrentstockUpdatePrice = dfCurrentstockUpdatePrice.tail(1)

            # SUMMARY
            if (dfCurrentstockUpdatePrice.shape[0]) > 0:
                priceForWeek = float(dfCurrentstockUpdatePrice['PRICE'].iloc[0])
                dqQty = dfStocksCurrentlyBought.query(queryGetCurrentStockUpdatePrice)
                weekQtyForUpdate = float(dqQty['QTY'].iloc[0])
                maxPriceTillNow = float(dqQty['MAXPRICE'].iloc[0])
                weekQtyForUpdate = float(dqQty['QTY'].iloc[0])
                updatePurchasePrice = float(dqQty['UPDATEPRICE'].iloc[0])
                updateCurrentPrice = float(dqQty['UPDATEPRICE'].iloc[0])
#                updateCurrentValue = float(dqQty['CURRVALUE'].iloc[0])

                if( maxPriceTillNow < priceForWeek):
                    maxPriceTillNow = priceForWeek
                    dfStocksCurrentlyBought = dfStocksCurrentlyBought.append({'STOCK': stockUpdatePrice, 'PURCHASEDATE': processingDate, 'PURCHASEPRICE': updatePurchasePrice, 'QTY': weekQtyForUpdate, 'UPDATEDATE' : processingDate,'UPDATEPRICE': updateCurrentPrice, 'MAXPRICE': maxPriceTillNow, 'ROC': ROI, 'ACTIVE': 'X'}, ignore_index=True)



######################----------------------###################################
######## SELL STOCK LOGIC #####################################################
######################----------------------###################################
        dfstocksActive = dfStocksCurrentlyBought.query(checkActive())
        stockBoughtList = dfstocksActive.STOCK.unique()

        for stockToSellCheck in stockBoughtList:

            dfStockWithCurrentDate = dfstocks.query(createqueryDate(processingDate))
            queryGetCurrentPriceWithDateSTOCK = checkStock(stockToSellCheck)
            dfStockWithCurrentDate = dfStockWithCurrentDate.query(queryGetCurrentPriceWithDateSTOCK)
# SELLING RULES
# 1)If increasing index this week : current price is higher than 0.8 MAX PRICE => sell
# 2) If currennt price or max price is lowe than 0.9 max price => sell
            listMaxStockPrice = dfStocksCurrentlyBought.groupby(['STOCK'])['MAXPRICE'].max()
            stockPriceThisStock =  round(float (listMaxStockPrice.get(key=stockToSellCheck)),2)
            # Get current price if it's less than 0.8 MAX PRICE if index increasing or less than 0.9 index decreasing
            if dfStockWithCurrentDate.size > 0:
                currentStockPrice = dfStockWithCurrentDate.iloc[-1]['PRICE']
                dfCurrentStockData = dfStocksCurrentlyBought.query(queryGetCurrentPriceWithDateSTOCK )
                STOCKQTY = dfCurrentStockData.iloc[-1]['QTY']
                if (isIndexIncreasing ==1):
                    stopLossPrice = c_stoplossfactorindexdown *  stockPriceThisStock
                else:
                    stopLossPrice = c_stoplossindexup *  stockPriceThisStock

                if (currentStockPrice < stopLossPrice):
                    dfStocksCurrentlyBought.loc[ (dfStocksCurrentlyBought.STOCK == stockToSellCheck  ),'ACTIVE'] = ''
                    print("SELL : STOCK VALUE IS : " + str(stockToSellCheck) +' - ' + str(processingDate) +' - ' +  ' Total Amount : ' + fround(currentStockPrice * STOCKQTY ))
                    totalStocksHeld = totalStocksHeld - 1
                    totalStopLossMoney = totalStopLossMoney + currentStockPrice * STOCKQTY
                    totalValueSell = totalValueSell + totalStopLossMoney

## BUY RULES

    if (isIndexIncreasing == 1):
        for  stock in companiesList :
            queryGetCurrentStockData = 'STOCK' + '==' + "'" + stock + "'"
            dfCurrentStock = dfstocks.query(queryGetCurrentStockData)
            periodToLookBack = 20 # 20 weeks
            queryDateForCurrentStock = "DATE" + "<" + "'" + processingDate + "'"
            dfCurrentStocklookBackData = dfCurrentStock.query(queryDateForCurrentStock)
            dfCurrentStocklookBackData = dfCurrentStocklookBackData.tail(periodToLookBack)
            if dfCurrentStocklookBackData.shape[0] > periodToLookBack - 1:
                start = 0
                length = periodToLookBack - 1
                stockHighestPointValid = True
                latestpriceIndex = length - start
                for n in range(start, start + length):
                    currentweekPrice  = dfCurrentStocklookBackData['PRICE'].iloc[latestpriceIndex]
                    lastweekPrice = dfCurrentStocklookBackData['PRICE'].iloc[n]
                    if (lastweekPrice > currentweekPrice):
                        stockHighestPointValid = False
                        break

                if( stockHighestPointValid == True ):
                    STOCK = dfCurrentStocklookBackData['STOCK'].iloc[-1]
                    VOLUME = dfCurrentStocklookBackData['VOLUME'].iloc[-1]
                    DATE = processingDate #dfCurrentStocklookBackData['DATE'].iloc[-1]
                    ROI = dfCurrentStocklookBackData['ROC100'].iloc[-1]
## how to use this .it should be the same as last price
                    maxPrice = dfCurrentStocklookBackData.loc[dfCurrentStocklookBackData['PRICE'].idxmax()]['PRICE']

                    if (ROI > 150 and VOLUME > 1000000 and maxPrice > 5 and stockHighestPointValid == True):

                        dfCheckStockALreadyAdded = dfStocksCurrentlyBought.query(queryGetCurrentStockData)
                        if dfCheckStockALreadyAdded.size == 0:
                            totalStocksHeld = totalStocksHeld + 1
                            CURRENTPRICESTOCKDATE = ''
                            numberOfStocks = int(moneyforeachstock / maxPrice)
                            amountSpent = numberOfStocks *  maxPrice
                            try:
                                currentPriceAPICall = str(fround(yf.Ticker(STOCK).info['regularMarketPrice']))
                                currentValueAPICall = str(fround( yf.Ticker(STOCK).info['regularMarketPrice'] * numberOfStocks))
                            except Exception as e:                                
                                print('No current value found for ' + STOCK)
                            print( str(totalStocksHeld) +' Buy : ' + STOCK + ',' + str(processingDate) + ',' + 'Purch Price: '+ fround(maxPrice) + ',Curr Price:'+ currentPriceAPICall + ',' +'QTY: '+ str(numberOfStocks) +','+ fround(amountSpent) +','+ 'Curr Value: '+ currentValueAPICall )
                            dfStocksCurrentlyBought = dfStocksCurrentlyBought.append({'STOCK':STOCK , 'PURCHASEDATE' : DATE , 'PURCHASEPRICE': maxPrice , 'QTY': numberOfStocks , 'CURRVALUE': currentValueAPICall ,'CURRPRICE': currentPriceAPICall , 'MAXPRICE' : maxPrice, 'ROC': ROI , 'ACTIVE': 'X' }, ignore_index=True)
                            totalSpent = totalSpent + amountSpent
    else:
        print('NO RECOMMENDATIONS AS INDEX IS GOING DOWN : ' + str(start_date))


# Update MAX Price of this week
    if (isIndexIncreasing == 1):
        stockWeekList = dfStocksCurrentlyBought.query("ACTIVE=='X'").STOCK.unique()
        for stockWeekValue in stockWeekList:
            queryGetCurrentStockDataWeekValue = 'STOCK' + '=='+"'"+stockWeekValue+"'"
            dfCurrentstockWeekValue=dfstocks.query(queryGetCurrentStockDataWeekValue)

            queryGetCurrentPriceWithDateWeekValue="DATE"+"=="+"'"+processingDate+"'"
            dfCurrentstockWeekValue=dfCurrentstockWeekValue.query(queryGetCurrentPriceWithDateWeekValue)
            dfCurrentstockWeekValue=dfCurrentstockWeekValue.tail(1)


            #SUMMARY
            if((dfCurrentstockWeekValue.shape[0])>0 and (processingDate == end_date)):
                priceForWeek=float(dfCurrentstockWeekValue['PRICE'].iloc[0])
                dqQty = dfStocksCurrentlyBought.query(queryGetCurrentStockDataWeekValue)
                weekQty=float(dqQty['QTY'].iloc[0])
                stockValue=float(priceForWeek*weekQty)
                weektotalValue = weektotalValue + stockValue

                weekPriceAPICall = str(fround(yft.Ticker(stockWeekValue).info['regularMarketPrice']))
                weekValueAPICall = str(fround( yft.Ticker(stockWeekValue).info['regularMarketPrice'] * weekQty))
                weektotalValueAPI = weektotalValueAPI + float(weekValueAPICall)
                currWeekROI = fround((weektotalValue -totalSpent)/totalSpent*100)
                currPriceAPI = fround((weektotalValueAPI -totalSpent)/totalSpent*100)
                print('Stock: ' + stockWeekValue +", Curr Value: " + weekValueAPICall +", Stock Price: "+ weekPriceAPICall + ",QTY: " +str(weekQty))

        if( weektotalValue > 0):
            print('Spent:'+fround(totalSpent)+'-'+'Week Value @ ' + processingDate  +'- '+ 'Week Value: '+ fround(totalValueSell + weektotalValue) + 'Curr Price Week Value : ' + fround(weektotalValueAPI) +' ROI%='+ currWeekROI +'%'  +' CURR ROI%='+ currPriceAPI +'%' )



    start_date += delta


print("--- %s seconds ---" % (time.time() - start_time))






Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Index : 2019-07-05 - 1
1 Buy : fate,2019-07-05,Purch Price: 20.3,Curr Price:24.68,QTY: 123,2496.9,Curr Value: 3035.64
2 Buy : ntra,2019-07-05,Purch Price: 27.58,Curr Price:37.26,QTY: 90,2482.2,Curr Value: 3353.4
3 Buy : run,2019-07-05,Purch Price: 18.76,Curr Price:16.83,QTY: 133,2495.08,Curr Value: 2238.39
4 Buy : arql,2019-07-05,Purch Price: 11.01,Curr Price:20,QTY: 227,2499.27,Curr Value: 4540
5 Buy : enph,2019-07-05,Purch Price: 18.23,Curr Price:30.97,QTY: 137,2497.51,Curr Value: 4242.89
6 Buy : iova,2019-07-05,Purch Price: 24.52,Curr Price:23.97,QTY: 101,2476.52,Curr Value: 2420.97
7 Buy : vcyt,2019-07-05,Purch Price: 28.51,Curr Price:30.48,QTY: 87,2480.37,Curr Value: 2651.76
8 Buy : srpt,2019-07-05,Purch Price: 151.95,Curr Price:130.25,QTY: 16,2431.2,Curr Value: 2084.0
9 Buy : mrtx,2019-07-05,Purch Price: 103.0,Curr Price:97.74,QTY: 24,2472.0,Curr Value: